In [1]:
# coding: utf-8

import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [2]:
link = 'http://piaui.folha.uol.com.br/'

In [3]:
req = requests.get(link)

In [4]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='main-content')
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='bloco size-1 destaque ')
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='bloco size-2 ')
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='inner')

In [ ]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

https://piaui.folha.uol.com.br/a-revista/
100% [..............................................................................] 38296 / 38296['revista piauí']
news_in_db: False
 -- no categories -- 
https://piaui.folha.uol.com.br/sao-particulares-diz-exercito-sobre-caminhoes-usados-em-campanha/
100% [..............................................................................] 79015 / 79015['“São particulares”, diz Exército sobre caminhões usados em campanha']
news_in_db: False
 -- no categories -- 
https://piaui.folha.uol.com.br/hungria-franca-e-brasil/
100% [..............................................................................] 62075 / 62075['Hungria, França e Brasil']
news_in_db: False
 -- no categories -- 
https://piaui.folha.uol.com.br/plata-o-plomo/
100% [..............................................................................] 58833 / 58833['Plata o plomo']
news_in_db: False
 -- no categories -- 
https://piaui.folha.uol.com.br/e-se-bolsonaro-ganhar/
100% [......

100% [............................................................................] 284204 / 284204['Bolsonaro não queria sair da Santa Casa']
news_in_db: False
 -- no categories -- 
https://piaui.folha.uol.com.br/sem-elite-sem-quase-nada/
100% [............................................................................] 239694 / 239694['Sem a elite, sem (quase) nada']
news_in_db: False
 -- no categories -- 
https://piaui.folha.uol.com.br/por-que-mulheres-trocaram-marina-por-haddad-ciro-e-bolsonaro/
100% [............................................................................] 145995 / 145995['Por que mulheres trocaram Marina por Haddad, Ciro e Bolsonaro']
news_in_db: False
 -- no categories -- 
https://piaui.folha.uol.com.br/antipetismo-e-democracia/
100% [..............................................................................] 62066 / 62066['Antipetismo e democracia']
news_in_db: False
 -- no categories -- 
https://piaui.folha.uol.com.br/materia/o-fiador/
100% [.........

In [ ]:
noticias[0]